In [88]:
import pyodbc
import re

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

from sqlalchemy import create_engine
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_similarity

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from googletrans import Translator

from transform_copy import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eats\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [18]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [671]:
pd.DataFrame(engine.execute(
    """
    SELECT * FROM ApplicantExperience
    """
)).columns

Index(['ApplicantExperienceID', 'ApplicantID', 'DateTo', 'DateFrom',
       'Industry', 'IsUntilPresent', 'CompanyName', 'JobDescription',
       'Position', 'Salary', 'Supervisor', 'SupervisorContactNumber',
       'SupervisorEmail'],
      dtype='object')

Job

In [ ]:
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Applicant

In [ ]:
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

['', '', '', '', '', '', '', '', '']

['', '', '', '', '', '', '', '', '', '', '', '', '']

In [ ]:
'''
JOB>>>MATCH<<<APPLICANT

Dob == UsiaMax
SalaryMean == ExpectedSalary

CityName == CurrentCityName
ProvinceName == CurrentProvinceName
DriverLicenseType
EducationLevelID
Gender
MaritalStatus
MajorName
IsUsingGlasses

JobTile, FunctionPositionName == Position
Description, Requirement == JobDescription, Strengthness
'''

In [182]:
applicant_id = 31790

df_job = pd.DataFrame(engine.execute(
    """
    SELECT Job.JobID, Job.JobTitle, FunctionPosition.FunctionPositionName, EducationLevel.EducationLevelName, City.Name AS CityName, Province.Name AS ProvinceName, Major.MajorName, Job.Description, Job.Requirement
    FROM (((((Job
    RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
    RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN City ON Job.CityID = City.CityID)
    RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
    RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
    WHERE JobStatus='Publish'
    """
))

df_applicant = pd.DataFrame(engine.execute(
    f"""
    SELECT Applicant.ApplicantID, Applicant.Dob, Applicant.Strengthness, Applicant.Weaknesses, City.Name AS CityName, Province.Name AS ProvinceName
    FROM ((Applicant
    RIGHT JOIN City ON Applicant.CurrentAddressCityID = City.CityID)
    RIGHT JOIN Province ON Applicant.CurrentAddressProvinceID = Province.ProvinceID)
    WHERE ApplicantID={applicant_id}
    """
))

df_applicant_education = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantEducation.ApplicantID, ApplicantEducation.DateStart, ApplicantEducation.DateEnd, EducationLevel.EducationLevelName, Major.MajorName
    FROM ((ApplicantEducation
    RIGHT JOIN EducationLevel ON ApplicantEducation.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN Major ON ApplicantEducation.MajorID = Major.MajorID)
    WHERE ApplicantID={applicant_id}
    """
))

df_applicant_experience = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantExperience.ApplicantID, ApplicantExperience.DateFrom, ApplicantExperience.DateTo, ApplicantExperience.Industry, ApplicantExperience.JobDescription, ApplicantExperience.Position
    FROM ApplicantExperience
    WHERE ApplicantID={applicant_id}
    """
))

Applicant

In [183]:
df_applicant = df_applicant.drop_duplicates()
df_applicant = df_applicant.fillna('')
df_applicant_education = df_applicant_education.fillna('')
df_applicant_experience = df_applicant_experience.fillna('')

'''applicant'''
df_applicant['Age'] = pd.to_datetime(
    df_applicant.Dob.map(pick_date).apply(lambda x: filter_date(x, 1958, 2006))
).map(get_age)

df_applicant.drop(columns=['Dob'], inplace=True)

df_applicant.Age = df_applicant.Age.fillna(0).astype(int)

'''education'''
df_applicant_education.DateStart = pd.to_datetime(
    df_applicant_education.DateStart.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_education.DateEnd = pd.to_datetime(
    df_applicant_education.DateEnd.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_education = df_applicant_education[~(df_applicant_education.DateStart.isna()) & ~(df_applicant_education.DateEnd.isna())]
df_applicant_education = df_applicant_education.sort_values('DateStart').groupby(['ApplicantID']).agg('last')

df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)

'''experience'''
df_applicant_experience.DateFrom = pd.to_datetime(
    df_applicant_experience.DateFrom.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_experience.DateTo = pd.to_datetime(
    df_applicant_experience.DateTo.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_experience = df_applicant_experience[~(df_applicant_experience.DateFrom.isna()) & ~(df_applicant_experience.DateTo.isna())]

# add YearsOfExperience column
df_applicant_experience['YearsOfExperience'] = substract_months(
    df_applicant_experience.DateFrom, df_applicant_experience.DateTo
)

df_applicant_experience = df_applicant_experience.sort_values('DateFrom').groupby(['ApplicantID']).agg({
    'DateFrom': 'last',
    'DateTo': 'last',
    'Industry': ' '.join,
    'JobDescription': ' '.join,
    'Position': ' '.join,
    'YearsOfExperience': 'sum',
})

df_applicant_experience.drop(columns=['DateFrom', 'DateTo'], inplace=True)

'''merge'''
df_applicant = pd.merge(df_applicant, df_applicant_experience, on=['ApplicantID'])
df_applicant = pd.merge(df_applicant, df_applicant_education, on=['ApplicantID'])

'''remove weaknesses'''
df_applicant = df_applicant.drop(columns=['Weaknesses'])

'''preprocessing'''
df_applicant.set_index(['ApplicantID'], inplace=True)

df_applicant[df_applicant.select_dtypes(object).columns] = df_applicant[df_applicant.select_dtypes(object).columns].applymap(str.lower)

for col in ['Strengthness', 'JobDescription', 'Industry', 'Position']:
    df_applicant[col] = df_applicant[col].map(clean_text)

'''translate'''
translator = Translator(service_urls=['translate.googleapis.com'])
for col in ['Strengthness', 'Industry', 'JobDescription', 'Position']:
    df_applicant[col] = df_applicant[col].apply(lambda x: translator.translate(x, dest='id').text)


In [184]:
df_applicant

,Strengthness,CityName,ProvinceName,Age,Industry,JobDescription,Position,YearsOfExperience,EducationLevelName,MajorName
ApplicantID,,,,,,,,,,
31790,pandai mengoperasikan multitasking komputer da...,surabaya,jawa timur,25,pendidikan pelayanan kesehatan,les privat siswa sd smp bagian jasa pelayanan ...,guru les privat mahasiswa magang,2,s1,akuntansi


Job

In [185]:
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

df_job = df_job.applymap(str.lower)
df_job.EducationLevelName = df_job.EducationLevelName.replace('none', '')

translator = Translator(service_urls=['translate.googleapis.com'])

df_job.JobTitle = df_job.JobTitle.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.FunctionPositionName = df_job.FunctionPositionName.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.MajorName = df_job.MajorName.apply(lambda x: translator.translate(x, dest='id').text.lower())

df_job.Description = df_job.Description.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.Requirement = df_job.Requirement.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())

In [258]:
df_job.head(2)

,JobTitle,FunctionPositionName,EducationLevelName,CityName,ProvinceName,MajorName,Description,Requirement,Similarity
JobID,,,,,,,,,
2775,sekretaris direksi,sekretaris,,surabaya,jawa timur,sekretaris,melakukan aktivitas kesekretariatan perusahaan...,usia maksimal 35 tahun pendidikan minimal s1 j...,0.176869
2785,staff desain,grafis desain,s1,jakarta,dki jakarta,desain grafis,membuat desain yang menarik untuk kebutuhan pe...,usia max 30 tahun pendidikan minimal s1 desain...,0.057073


In [ ]:
import tensorflow_datasets as tfds

ratings = tfds.load("movielens/100k-ratings", split="train")

In [ ]:
df_job.columns

In [189]:
ds_job = tf.data.Dataset.from_tensor_slices(dict(df_job))

In [617]:
class JobModel:
    def __init__(self, user_text, output_mode='multi_hot', length=5):
        if output_mode == 'int':
            self.job_title = tf.keras.layers.TextVectorization(max_tokens=1_000, output_mode=output_mode, output_sequence_length=length)
        else:
            self.job_title = tf.keras.layers.TextVectorization(max_tokens=1_000, output_mode=output_mode)
        self.job_title.adapt(ds_job.map(lambda x: x['JobTitle']))

        self.job_title_embedding = tf.keras.layers.Embedding(
            input_dim=self.job_title.vocabulary_size(),
            output_dim=32,
        )

        self.job_title_model = tf.keras.Sequential([self.job_title, self.job_title_embedding])

        # job_vector = job_title_model(np.array(df_job.JobTitle.values))
        # user_vector = job_title_model(np.array(['asisten direksi']))

        self.job_vector = self.job_title(np.array(df_job.JobTitle.values))
        self.user_vector = self.job_title(np.array([user_text]))

In [605]:
txt = 'staf keuangan'
mod = ['multi_hot', 'count', 'tf_idf']

for i in mod:
    model = JobModel(txt, i)
    df_job[str('Similarity ' + i)] = cosine_similarity(model.job_vector, model.user_vector)

In [ ]:
df_job[['JobTitle', 'Similarity multi_hot', 'Similarity count', 'Similarity tf_idf']].sort_values('Similarity tf_idf', ascending=False)

In [612]:
model.job_vector.shape, model.user_vector.shape

(TensorShape([42, 4]), TensorShape([1, 2]))

In [555]:
job_title = tf.keras.layers.TextVectorization(max_tokens=1_000, output_mode='tf_idf')
job_title.adapt(ds_job.map(lambda x: x['JobTitle']))

job_title_embedding = tf.keras.layers.Embedding(
    input_dim=job_title.vocabulary_size(),
    output_dim=32,
)

job_title_model = tf.keras.Sequential([job_title, job_title_embedding])

# job_vector = job_title_model(np.array(df_job.JobTitle.values))
# user_vector = job_title_model(np.array(['asisten direksi']))

job_vector = job_title(np.array(df_job.JobTitle.values))
user_vector = job_title(np.array(['supervisor proyek lapangan sekretaris manajer sekretaris']))

In [513]:
df_job['Similarity'] = cosine_similarity(job_vector, user_vector)

In [ ]:
df_job[['JobTitle', 'Similarity']].sort_values('Similarity', ascending=False)

In [56]:
'''encoding'''
job_title = tf.keras.layers.StringLookup()
job_title.adapt(ds_job.map(lambda x: x['JobTitle']))

job_function = tf.keras.layers.StringLookup()
job_function.adapt(ds_job.map(lambda x: x['FunctionPositionName']))

job_education = tf.keras.layers.StringLookup()
job_education.adapt(ds_job.map(lambda x: x['EducationLevelName']))

job_city = tf.keras.layers.StringLookup()
job_city.adapt(ds_job.map(lambda x: x['CityName']))

job_province = tf.keras.layers.StringLookup()
job_province.adapt(ds_job.map(lambda x: x['ProvinceName']))

job_major = tf.keras.layers.StringLookup()
job_major.adapt(ds_job.map(lambda x: x['MajorName']))

job_description = tf.keras.layers.TextVectorization(output_sequence_length=300)
job_description.adapt(ds_job.map(lambda x: x['Description']))

job_requirement = tf.keras.layers.TextVectorization()
job_requirement.adapt(ds_job.map(lambda x: x['Requirement']))


'''embedding'''
job_title_embedding = tf.keras.layers.Embedding(
    input_dim=job_title.vocabulary_size(),
    output_dim=32
)

job_function_embedding = tf.keras.layers.Embedding(
    input_dim=job_function.vocabulary_size(),
    output_dim=32
)

job_education_embedding = tf.keras.layers.Embedding(
    input_dim=job_education.vocabulary_size(),
    output_dim=32
)

job_city_embedding = tf.keras.layers.Embedding(
    input_dim=job_city.vocabulary_size(),
    output_dim=32
)

job_province_embedding = tf.keras.layers.Embedding(
    input_dim=job_province.vocabulary_size(),
    output_dim=32
)

job_major_embedding = tf.keras.layers.Embedding(
    input_dim=job_major.vocabulary_size(),
    output_dim=32
)

job_description_embedding = tf.keras.layers.Embedding(
    input_dim=job_description.vocabulary_size(),
    output_dim=32
)

job_requirement_embedding = tf.keras.layers.Embedding(
    input_dim=job_requirement.vocabulary_size(),
    output_dim=32
)


'''model'''
job_title_model = tf.keras.Sequential([job_title, job_title_embedding])
job_function_model = tf.keras.Sequential([job_function, job_function_embedding])
job_education_model = tf.keras.Sequential([job_education, job_education_embedding])
job_city_model = tf.keras.Sequential([job_city, job_city_embedding])
job_province_model = tf.keras.Sequential([job_province, job_province_embedding])
job_major_model = tf.keras.Sequential([job_major, job_major_embedding])
job_description_model = tf.keras.Sequential([job_description, job_description_embedding])
job_requirement_model = tf.keras.Sequential([job_requirement, job_requirement_embedding])

In [57]:
df_job.columns, df_applicant.columns

(Index(['JobTitle', 'FunctionPositionName', 'EducationLevelName', 'CityName',
        'ProvinceName', 'MajorName', 'Description', 'Requirement'],
       dtype='object'),
 Index(['Strengthness', 'CityName', 'ProvinceName', 'Age', 'Industry',
        'JobDescription', 'Position', 'YearsOfExperience', 'EducationLevelName',
        'MajorName'],
       dtype='object'))

In [58]:
df_job.loc[2785]

JobTitle                                                     staff desain
FunctionPositionName                                        grafis desain
EducationLevelName                                                     s1
CityName                                                          jakarta
ProvinceName                                                  dki jakarta
MajorName                                                   desain grafis
Description             membuat desain yang menarik untuk kebutuhan pe...
Requirement             usia max 30 tahun pendidikan minimal s1 desain...
Name: 2785, dtype: object

In [59]:
job_embedding = np.concatenate((
    job_title_model([df_job.loc[2785].JobTitle]),
    job_function_model([df_job.loc[2785].FunctionPositionName]),
    job_education_model([df_job.loc[2785].EducationLevelName]),
    job_city_model([df_job.loc[2785].CityName]),
    job_province_model([df_job.loc[2785].ProvinceName]),
    job_major_model([df_job.loc[2785].MajorName]),
    job_description_model([df_job.loc[2785].Description]),
    job_requirement_model([df_job.loc[2785].Requirement]),
))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 6 has 3 dimension(s)

In [64]:
cosine_sim = tf.keras.losses.CosineSimilarity(axis=1)

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(job_title)

model.predict(df_job.JobTitle.values)

In [14]:
# Define user and job data
user_text = "I am good at programming"
job_text = "We need a programmer"
user_cat = ["Data Science", "Programming"]
job_cat = ["Programming", "Web Development"]
user_num = [5, 7, 9]
job_num = [3, 8, 7]

text_vector = tf.keras.layers.TextVectorization(output_sequence_length=10)
text_vector.adapt(job_text.split(' '))

text_model = tf.keras.Sequential([
    text_vector,
    tf.keras.layers.Embedding(
        input_dim=text_vector.vocabulary_size(),
        output_dim=10
    )
])

user_text_embedding = text_model.predict(user_text.split(' '))

1/1 [==============================] - 0s 42ms/step


In [15]:
user_text_embedding

array([[[-0.00855603, -0.04862776,  0.00302802, -0.02810311,
         -0.04780581, -0.02766582,  0.0056753 ,  0.02541116,
         -0.02893522,  0.02540194],
        [ 0.01162858,  0.04382291,  0.02898641, -0.01834045,
         -0.04317871,  0.00940644, -0.03661983, -0.03443255,
         -0.03611128, -0.02730765],
        [ 0.01162858,  0.04382291,  0.02898641, -0.01834045,
         -0.04317871,  0.00940644, -0.03661983, -0.03443255,
         -0.03611128, -0.02730765],
        [ 0.01162858,  0.04382291,  0.02898641, -0.01834045,
         -0.04317871,  0.00940644, -0.03661983, -0.03443255,
         -0.03611128, -0.02730765],
        [ 0.01162858,  0.04382291,  0.02898641, -0.01834045,
         -0.04317871,  0.00940644, -0.03661983, -0.03443255,
         -0.03611128, -0.02730765],
        [ 0.01162858,  0.04382291,  0.02898641, -0.01834045,
         -0.04317871,  0.00940644, -0.03661983, -0.03443255,
         -0.03611128, -0.02730765],
        [ 0.01162858,  0.04382291,  0.02898641, -0.0

In [7]:
text_vector.get_vocabulary()

['', '[UNK]', 'we', 'programmer', 'need', 'a']